In [170]:
import numpy as np
import xgboost as xgb
import cryptoaml.datareader as cdr
from sklearn.metrics import f1_score

In [171]:
elliptic = cdr.get_data("elliptic")
data = elliptic.train_test_split(train_size=0.7, 
                                 feat_set="AF", 
                                 inc_meta=False,
                                 inc_unknown=False)

train_data = data.train_X
train_data["class"] = data.train_y
test_data = data.test_X
test_data["class"] = data.test_y 

In [178]:
class AdaptiveStackedBoostClassifier():
    def __init__(self,
                 n_estimators=5,
                 window_size=2000,
                 verbose=True):
        self._first_run = True
        self._n_estimators = n_estimators
        self._window_size = window_size
        self._X_buffer = np.array([])
        self._y_buffer = np.array([])
        self._meta_learner = xgb.XGBClassifier()
        
        # 3*N matrix 
        # 1st row - first level models
        # 2nd row - number of training rounds 
        # 3rd row - performance in terms on feature importance of meta model for n previous rounds
        self._ensemble_matrix = [[None for x in range(n_estimators)] for y in range(3)]
        self._ensemble_size = 0 
        
    def partial_fit(self, X, y):
        if self._first_run:
            self._X_buffer = np.array([]).reshape(0, X.shape[1])
            self._y_buffer = np.array([])
            self._first_run = False
                           
        self._X_buffer = np.concatenate((self._X_buffer, X))
        self._y_buffer = np.concatenate((self._y_buffer, y))
        while self._X_buffer.shape[0] >= self._window_size:
            self._train_on_mini_batch(X=self._X_buffer[0:self._window_size, :],
                                      y=self._y_buffer[0:self._window_size])
            delete_idx = [i for i in range(self._window_size)]
            self._X_buffer = np.delete(self._X_buffer, delete_idx, axis=0)
            self._y_buffer = np.delete(self._y_buffer, delete_idx, axis=0)
    
    def _train_on_mini_batch(self, X, y):
        
        print("-----------")
        
        # split mini batch to train ensemble and meta learner 
        n_instances = X.shape[0]
        n_instances_first = int(n_instances * 0.7)
        X_first = X[0: n_instances_first, :]
        y_first = y[0: n_instances_first]
        X_second = X[n_instances_first:n_instances, :]
        y_second = y[n_instances_first:n_instances]
          
        new_model_idx = self._ensemble_size 
        ensemble_full = self._ensemble_size == self._n_estimators 
        worst_score = 1 
        worst_model = None 
        
        # continue fitting model in ensemble 
        meta_model_X = []
        if self._ensemble_size > 0:
            print("SIZE: {}".format(self._ensemble_size))
            performances = self._meta_learner.feature_importances_
            for i in range(self._ensemble_size):
                
                # partially fit and construct features for meta model
                old_model = self._ensemble_matrix[0][i]
                old_model_booster = old_model.get_booster()
                old_model.fit(X_first, y_first, xgb_model=old_model_booster)
                y_pred_prob_old_model = old_model.predict_proba(X_second) 
                print("PROB B: {}".format(y_pred_prob_old_model[0]))

                # construct features from probs for meta model 
                meta_model_X = y_pred_prob_old_model if i == 0 else np.hstack((meta_model_X, y_pred_prob_old_model))                    
                                
                # update model in matrix 
                self._ensemble_matrix[0][i] = old_model # update model 
                last_performance = performances[i * 2] + performances[(i*2)+1]  
                current_round =  self._ensemble_matrix[1][i]
                self._ensemble_matrix[2][i][current_round % self._n_estimators] = last_performance                

                if ensemble_full and current_round >= 5:
#                     print("IM HERE")
                    if self._ensemble_matrix[2][i].sum() <= worst_score:
                        worst_score = self._ensemble_matrix[2][i].sum()
                        new_model_idx = i
                        print("WORST {} ".format(i))
                    
                self._ensemble_matrix[1][i] += 1
                     
#         print(new_model_idx)
        
        # check if ensemble is full 
        if ensemble_full:
            print("ensemble is full")
            
        # partially train a new model to become part of the ensemble 
        new_model = xgb.XGBClassifier()
        new_model.fit(X_first, y_first)
        y_pred_prob = new_model.predict_proba(X_second)
        print("PROB A: {}".format(y_pred_prob[0]))
        if self._ensemble_size == 0:
            meta_model_X = y_pred_prob
        else: 
            if ensemble_full:
                print("")
            else:
                meta_model_X = np.hstack((meta_model_X, y_pred_prob))      
            
        # fit meta model  
        n_current_meta_model_X = meta_model_X.shape[1]  
        n_meta_model_X = self._n_estimators * 2
        if n_current_meta_model_X < n_meta_model_X:
            n_diff = n_meta_model_X - n_current_meta_model_X
            fillers = np.zeros((meta_model_X.shape[0], n_diff))
            meta_model_X = np.hstack((meta_model_X, fillers)) 
                
#         print(meta_model_X.shape)
        if self._ensemble_size == 0:
            self._meta_learner.fit(meta_model_X, y_second)
        else:
            tmp_meta_learner_booster = self._meta_learner.get_booster()
            self._meta_learner.fit(meta_model_X, y_second, xgb_model=tmp_meta_learner_booster)    
        
        print(new_model_idx)
        self._ensemble_matrix[0][new_model_idx] = new_model 
        self._ensemble_matrix[1][new_model_idx] = 0 
        self._ensemble_matrix[2][new_model_idx] = np.zeros(self._n_estimators) 
        
        if self._ensemble_size != self._n_estimators:
            self._ensemble_size += 1 
               
    def predict(self, X):
      
        # only one model in ensemble use its predictions 
        if self._ensemble_size == 1:
            return self._ensemble_matrix[0][0].predict(X)
        
        # predict via meta learner 
        meta_model_X = []       
        # construct stracked features
        for i in range(self._ensemble_size):
            y_pred_prob_tmp_model = self._ensemble_matrix[0][i].predict_proba(X) 
            if i == 0:
                meta_model_X = y_pred_prob_tmp_model
                continue
            np.hstack((meta_model_X, y_pred_prob_tmp_model)) 
        
        # add fillers if needed 
        n_current_meta_model_X = meta_model_X.shape[1]  
        n_meta_model_X = self._n_estimators * 2
        if n_current_meta_model_X < n_meta_model_X:
            n_diff = n_meta_model_X - n_current_meta_model_X
            fillers = np.zeros((meta_model_X.shape[0], n_diff))
            meta_model_X = np.hstack((meta_model_X, fillers)) 
        
        return self._meta_learner.predict(meta_model_X)
        
        
    
#         print("NOW")
#         print(meta_importance)
        
# #         print(meta_importance)
#         if self._ensemble_size > 0:
#             for i in range(self._ensemble_size):
#                 f_idx = i * 2
#                 new_measure = meta_importance[f_idx] + meta_importance[f_idx+1]
#                 current_round = self._ensemble_matrix[1][i]
                
#                 # TODO -> CHECK THIS CURRENT ROUND THING 
#                 self._ensemble_matrix[2][i][current_round - 1 % self._n_estimators] = new_measure

#                 print(new_measure)
    
#         print("TRAIN")
#         print(self._ensemble_matrix[1])
#         print(self._ensemble_matrix[2])



incremental_xgb = xgb.XGBClassifier()
adpBoost = AdaptiveStackedBoostClassifier()

for ts in np.arange(train_data["ts"].min(), train_data["ts"].max()):
#     print("Start")
    train_set = train_data[train_data["ts"] == ts]
    train_set_X = train_set.iloc[:,:-1]
    train_set_y = train_set["class"]      
        
    test_set = train_data[train_data["ts"] == ts + 1]
    test_set_X = test_set.iloc[:,:-1].values
    test_set_y = test_set["class"].values
   
    adpBoost.partial_fit(train_set_X.values, train_set_y.values)
    
#     y_pred = adpBoost.predict(test_set_X)
#     evaluation = f1_score(test_set_y, y_pred, average='binary')
#     print("Proposed TS {}: {}".format(ts+1, evaluation))
            
#     if ts == 1:
#         incremental_xgb.fit(train_set_X.values, train_set_y.values)
#     else:
#         booster = incremental_xgb.get_booster()
#         incremental_xgb.fit(train_set_X.values, train_set_y.values, xgb_model=booster)
        
#     y_pred = incremental_xgb.predict(test_set_X)
#     evaluation = f1_score(test_set_y, y_pred, average='binary')
#     print("Incremental TS {}: {}".format(ts+1, evaluation))
    
    
# train_data_tmp = train_data[train_data["ts"] <= 34]


-----------
PROB A: [0.9989837  0.00101632]
0
-----------
SIZE: 1
PROB B: [9.9987447e-01 1.2552003e-04]
PROB A: [9.9965274e-01 3.4728620e-04]
1
-----------
SIZE: 2
PROB B: [9.9960059e-01 3.9939774e-04]
PROB B: [9.9922591e-01 7.7409094e-04]
PROB A: [9.9981189e-01 1.8813132e-04]
2
-----------
SIZE: 3
PROB B: [0.9849953  0.01500468]
PROB B: [0.94188005 0.05811994]
PROB B: [0.99391216 0.00608784]
PROB A: [0.99446124 0.00553876]
3
-----------
SIZE: 4
PROB B: [0.98452944 0.01547055]
PROB B: [0.994727 0.005273]
PROB B: [0.9789501  0.02104991]
PROB B: [0.81304264 0.18695739]
PROB A: [0.9928846  0.00711542]
4
-----------
SIZE: 5
PROB B: [0.9988693  0.00113071]
PROB B: [9.9985301e-01 1.4697922e-04]
PROB B: [9.999096e-01 9.041272e-05]
PROB B: [0.76241195 0.23758803]
PROB B: [0.9986924  0.00130761]
ensemble is full
PROB A: [0.9978377  0.00216227]

5


IndexError: list assignment index out of range